In [1]:
import ee

# Reset Earth Engine completely
ee.Reset()

# Initialize with standard (normal) endpoint
ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [2]:
# Earth Engine and Common Libraries
import ee
from pathlib import Path

# Authenticate and initialize Earth Engine
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')  # Try to use existing credentials first
except Exception:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [3]:
!pip install --upgrade --pre openforis-whisp




[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Check which endpoint is now active
print("EE Data Base URL:", ee.data._cloud_api_base_url)
print("EE API Base URL:", ee.data._api_base_url)

# Check if using standard endpoint
if 'highvolume' in str(ee.data._cloud_api_base_url):
    print("❌ Still using HIGH-VOLUME endpoint")
else:
    print("✅ Now using STANDARD endpoint")

EE Data Base URL: https://earthengine-highvolume.googleapis.com
EE API Base URL: https://earthengine-highvolume.googleapis.com/api
❌ Still using HIGH-VOLUME endpoint


In [5]:

import geopandas as gpd  # for random polygon generation in tests
import random  # for random polygon generation in tests
import math  # for random polygon generation in tests
import numpy as np  # for random polygon generation in tests
from shapely.geometry import Polygon  # for random polygon generation in tests
from shapely.validation import make_valid
from shapely.geometry import mapping  # for random polygon generation in tests

def generate_random_polygon(
    min_lon, min_lat, max_lon, max_lat, min_area_ha=1, max_area_ha=10, vertex_count=20
):
    """
    Generate a random polygon within bounds with approximate area in the specified range.
    Uses a robust approach that works well with high vertex counts and never falls back to squares.

    Args:
        min_lon, min_lat, max_lon, max_lat: Boundary coordinates
        min_area_ha: Minimum area in hectares
        max_area_ha: Maximum area in hectares
        vertex_count: Number of vertices for the polygon
    """
    # Initialize variables to ensure they're always defined
    poly = None
    actual_area_ha = 0

    # Simple function to approximate area in hectares (much faster)
    def approximate_area_ha(polygon, center_lat):
        # Get area in square degrees
        area_sq_degrees = polygon.area

        # Approximate conversion factor from square degrees to hectares
        # This varies with latitude due to the Earth's curvature
        lat_factor = 111320  # meters per degree latitude (approximately)
        lon_factor = 111320 * math.cos(
            math.radians(center_lat)
        )  # meters per degree longitude

        # Convert to square meters, then to hectares (1 ha = 10,000 sq m)
        return area_sq_degrees * lat_factor * lon_factor / 10000

    # Target area in hectares
    target_area_ha = random.uniform(min_area_ha, max_area_ha)

    # Select a center point within the bounds
    center_lon = random.uniform(min_lon, max_lon)
    center_lat = random.uniform(min_lat, max_lat)

    # Initial size estimate (in degrees)
    # Rough approximation: 0.01 degrees ~ 1km at equator
    initial_radius = math.sqrt(target_area_ha / (math.pi * 100)) * 0.01

    # Avoid generating too many points initially - cap vertex count for stability
    effective_vertex_count = min(
        vertex_count, 100
    )  # Cap at 100 to avoid performance issues

    # Primary approach: Create polygon using convex hull approach
    for attempt in range(5):  # First method gets 5 attempts
        try:
            # Generate random points in a circle around center with varying distance
            thetas = np.linspace(0, 2 * math.pi, effective_vertex_count, endpoint=False)

            # Add randomness to angles - smaller randomness for higher vertex counts
            angle_randomness = min(0.2, 2.0 / effective_vertex_count)
            thetas += np.random.uniform(
                -angle_randomness, angle_randomness, size=effective_vertex_count
            )

            # Randomize distances from center - less extreme for high vertex counts
            distance_factor = min(0.3, 3.0 / effective_vertex_count) + 0.7
            distances = initial_radius * np.random.uniform(
                1.0 - distance_factor / 2,
                1.0 + distance_factor / 2,
                size=effective_vertex_count,
            )

            # Convert to cartesian coordinates
            xs = center_lon + distances * np.cos(thetas)
            ys = center_lat + distances * np.sin(thetas)

            # Ensure points are within bounds
            xs = np.clip(xs, min_lon, max_lon)
            ys = np.clip(ys, min_lat, max_lat)

            # Create vertices list
            vertices = list(zip(xs, ys))

            # Close the polygon
            if vertices[0] != vertices[-1]:
                vertices.append(vertices[0])

            # Create polygon
            poly = Polygon(vertices)

            # Ensure it's valid
            if not poly.is_valid:
                poly = make_valid(poly)
                if poly.geom_type != "Polygon":
                    # If not a valid polygon, we'll try again
                    continue

            # Calculate approximate area
            actual_area_ha = approximate_area_ha(poly, center_lat)

            # Check if within target range
            if min_area_ha * 0.8 <= actual_area_ha <= max_area_ha * 1.2:
                return poly, actual_area_ha

            # Adjust size for next attempt based on ratio
            if actual_area_ha > 0:  # Avoid division by zero
                scale_factor = math.sqrt(target_area_ha / actual_area_ha)
                initial_radius *= scale_factor

        except Exception as e:
            print(f"Error in convex hull method (attempt {attempt+1}): {e}")

    # Second approach: Star-like pattern with controlled randomness
    # This is a fallback that will still create an irregular polygon, not a square
    for attempt in range(5):  # Second method gets 5 attempts
        try:
            # Use fewer vertices for stability in the fallback
            star_vertex_count = min(15, vertex_count)
            vertices = []

            # Create a star-like pattern with two radiuses
            for i in range(star_vertex_count):
                angle = 2 * math.pi * i / star_vertex_count

                # Alternate between two distances to create star-like shape
                if i % 2 == 0:
                    distance = initial_radius * random.uniform(0.7, 0.9)
                else:
                    distance = initial_radius * random.uniform(0.5, 0.6)

                # Add some irregularity to angles
                angle += random.uniform(-0.1, 0.1)

                # Calculate vertex position
                lon = center_lon + distance * math.cos(angle)
                lat = center_lat + distance * math.sin(angle)

                # Ensure within bounds
                lon = min(max(lon, min_lon), max_lon)
                lat = min(max(lat, min_lat), max_lat)

                vertices.append((lon, lat))

            # Close the polygon
            vertices.append(vertices[0])

            # Create polygon
            poly = Polygon(vertices)
            if not poly.is_valid:
                poly = make_valid(poly)
                if poly.geom_type != "Polygon":
                    continue

            actual_area_ha = approximate_area_ha(poly, center_lat)

            # We're less picky about size at this point, just return it
            if actual_area_ha > 0:
                return poly, actual_area_ha

            # Still try to adjust if we get another attempt
            if actual_area_ha > 0:
                scale_factor = math.sqrt(target_area_ha / actual_area_ha)
                initial_radius *= scale_factor

        except Exception as e:
            print(f"Error in star pattern method (attempt {attempt+1}): {e}")

    # Last resort - create a perturbed circle (never a square)
    try:
        # Create a circle-like shape with small perturbations
        final_vertices = []
        perturbed_vertex_count = 8  # Use a modest number for stability

        for i in range(perturbed_vertex_count):
            angle = 2 * math.pi * i / perturbed_vertex_count
            # Small perturbation
            distance = initial_radius * random.uniform(0.95, 1.05)

            # Calculate vertex position
            lon = center_lon + distance * math.cos(angle)
            lat = center_lat + distance * math.sin(angle)

            # Ensure within bounds
            lon = min(max(lon, min_lon), max_lon)
            lat = min(max(lat, min_lat), max_lat)

            final_vertices.append((lon, lat))

        # Close the polygon
        final_vertices.append(final_vertices[0])

        # Create polygon
        poly = Polygon(final_vertices)
        if not poly.is_valid:
            poly = make_valid(poly)

        actual_area_ha = approximate_area_ha(poly, center_lat)

    except Exception as e:
        print(f"Error in final fallback method: {e}")
        # If absolutely everything fails, create the simplest valid polygon (triangle)
        # This is different from a square and should be more compatible with your code
        offset = initial_radius / 2
        poly = Polygon(
            [
                (center_lon, center_lat + offset),
                (center_lon + offset, center_lat - offset),
                (center_lon - offset, center_lat - offset),
                (center_lon, center_lat + offset),
            ]
        )
        actual_area_ha = approximate_area_ha(poly, center_lat)

    # Return whatever we've created - never a simple square
    return poly, actual_area_ha


def generate_properties(area_ha, index):
    """
    Generate properties for features with sequential internal_id

    Args:
        area_ha: Area in hectares of the polygon
        index: Index of the feature to use for sequential ID
    """
    return {
        "internal_id": index + 1,  # Create sequential IDs starting from 1
    }


def create_geojson(
    bounds,
    num_polygons=25,
    min_area_ha=1,
    max_area_ha=10,
    min_number_vert=10,
    max_number_vert=20,
):
    """Create a GeoJSON file with random polygons within area range"""
    min_lon, min_lat, max_lon, max_lat = bounds
    # min_number_vert = 15
    # max_number_vert = 20

    features = []
    for i in range(num_polygons):
        # Random vertex count between 4 and 8
        # vertices = random.randint(4, 8)
        vertices = random.randint(min_number_vert, max_number_vert)

        # Generate polygon with area control
        polygon, actual_area = generate_random_polygon(
            min_lon,
            min_lat,
            max_lon,
            max_lat,
            min_area_ha=min_area_ha,
            max_area_ha=max_area_ha,
            vertex_count=vertices,
        )

        # Create GeoJSON feature with actual area
        properties = generate_properties(actual_area, index=i)
        feature = {
            "type": "Feature",
            "properties": properties,
            "geometry": mapping(polygon),
        }

        features.append(feature)

    # Create the GeoJSON feature collection
    geojson = {"type": "FeatureCollection", "features": features}

    return geojson


def reformat_geojson_properties(
    geojson_path,
    output_path=None,
    id_field="internal_id",
    start_index=1,
    remove_properties=False,
    add_uuid=False,
):
    """
    Add numeric IDs to features in an existing GeoJSON file and optionally remove properties.

    Args:
        geojson_path: Path to input GeoJSON file
        output_path: Path to save the output GeoJSON (if None, overwrites input)
        id_field: Name of the ID field to add
        start_index: Starting index for sequential IDs
        remove_properties: Whether to remove all existing properties (default: False)
        add_uuid: Whether to also add UUID field

    Returns:
        GeoDataFrame with updated features
    """

    # Read the GeoJSON
    # print(f"Reading GeoJSON file: {geojson_path}")
    gdf = gpd.read_file(geojson_path)

    # Remove existing properties if requested
    if remove_properties:
        # Keep only the geometry column and drop all other columns
        gdf = gdf[["geometry"]].copy()
        # print(f"Removed all existing properties from features")

    # Add sequential numeric IDs
    gdf[id_field] = [i + start_index for i in range(len(gdf))]

    # Optionally add UUIDs
    if add_uuid:
        gdf["uuid"] = [str(uuid.uuid4()) for _ in range(len(gdf))]

    # Write the GeoJSON with added IDs
    output_path = output_path or geojson_path
    gdf.to_file(output_path, driver="GeoJSON")
    print(f"Added {id_field} to GeoJSON and saved to {output_path}")

    return None


In [6]:
import ee
import openforis_whisp as whisp
import geopandas as gpd
import pandas as pd
import time
import threading
from queue import Queue
import logging
from typing import List, Optional, Dict, Any
from concurrent.futures import ThreadPoolExecutor, as_completed

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("whisp-high-volume")

# Optimized configuration for EE high-volume processing
EE_MAX_CONCURRENT = 10
EE_FEATURES_PER_BATCH = 50  # Features per Earth Engine request
MAX_RETRIES = 3
THREAD_POOL_SIZE = 4

class OptimizedWhispProcessor:
    """Optimized processor using Earth Engine high-volume patterns"""
    
    def __init__(self, max_concurrent=EE_MAX_CONCURRENT, features_per_batch=EE_FEATURES_PER_BATCH):
        self.max_concurrent = max_concurrent
        self.features_per_batch = features_per_batch
        self.semaphore = threading.Semaphore(max_concurrent)
        self.results = {}
        self.processing_stats = {'completed': 0, 'failed': 0, 'total': 0}
        
    def process_file_optimized(self, geojson_path: str, national_codes: Optional[List[str]] = None) -> pd.DataFrame:
        """Process file using optimized Earth Engine batching"""
        
        # Load the GeoDataFrame
        gdf = gpd.read_file(geojson_path)
        total_features = len(gdf)
        
        logger.info(f"Processing {total_features} features in batches of {self.features_per_batch}")
        
        # Split into feature batches
        feature_batches = []
        for i in range(0, total_features, self.features_per_batch):
            batch = gdf.iloc[i:i+self.features_per_batch]
            feature_batches.append(batch)
        
        logger.info(f"Created {len(feature_batches)} batches for Earth Engine processing")
        
        # Process batches concurrently using ThreadPoolExecutor
        results = []
        with ThreadPoolExecutor(max_workers=self.max_concurrent) as executor:
            # Submit all batches
            future_to_batch = {
                executor.submit(self._process_feature_batch, batch, national_codes, i): i 
                for i, batch in enumerate(feature_batches)
            }
            
            # Collect results as they complete
            for future in as_completed(future_to_batch):
                batch_idx = future_to_batch[future]
                try:
                    batch_result = future.result()
                    results.append(batch_result)
                    logger.info(f"Completed batch {batch_idx + 1}/{len(feature_batches)}")
                except Exception as e:
                    logger.error(f"Error processing batch {batch_idx + 1}: {e}")
                    self.processing_stats['failed'] += 1
        
        # Combine all results
        if results:
            combined_df = pd.concat(results, ignore_index=True)
            self.processing_stats['completed'] += len(results)
            logger.info(f"Successfully processed {len(combined_df)} total features")
            return combined_df
        else:
            logger.warning("No results to combine")
            return pd.DataFrame()
    
    def _process_feature_batch(self, batch_gdf: gpd.GeoDataFrame, national_codes: Optional[List[str]], batch_idx: int) -> pd.DataFrame:
        """Process a single batch of features using Earth Engine"""
        
        with self.semaphore:  # Limit concurrent EE requests
            # Convert GeoDataFrame to Earth Engine FeatureCollection directly
            features_list = []
            
            for idx, row in batch_gdf.iterrows():
                # Convert each feature to EE format
                geometry = row.geometry
                properties = {k: v for k, v in row.items() if k != 'geometry' and pd.notna(v)}
                
                # Convert shapely geometry to EE geometry
                if geometry is not None:
                    ee_geometry = ee.Geometry(geometry.__geo_interface__)
                    ee_feature = ee.Feature(ee_geometry, properties)
                    features_list.append(ee_feature)
            
            # Create FeatureCollection from features
            feature_collection = ee.FeatureCollection(features_list)
            
            # Process using optimized Earth Engine parameters
            return self._process_ee_feature_collection(feature_collection, national_codes, batch_idx)
    
    # def _process_ee_feature_collection(self, feature_collection: ee.FeatureCollection, 
    #                                  national_codes: Optional[List[str]], batch_idx: int) -> pd.DataFrame:
    #     """Process FeatureCollection using Earth Engine with retry logic"""
        
    #     for attempt in range(MAX_RETRIES):
    #         try:
    #             # Use whisp's Earth Engine processing with optimized parameters
    #             result_fc = whisp.whisp_stats_ee_to_ee(
    #                 feature_collection=feature_collection,
    #                 external_id_column=None,
    #                 national_codes=national_codes
    #             )
                
    #             # Configure Earth Engine computation for high-volume processing
    #             # Use higher tileScale and maxPixels for distributed computation
    #             result_dict = result_fc.getInfo()  # This triggers the computation
                
    #             # Convert EE result back to DataFrame
    #             df_result = self._ee_dict_to_dataframe(result_dict)
                
    #             # Add ISO2 country codes
    #             df_result = whisp.convert_iso3_to_iso2(
    #                 df=df_result,
    #                 iso3_column='iso3_country_code',
    #                 iso2_column='iso2_country_code'
    #             )
                
    #             logger.info(f"Batch {batch_idx + 1}: Processed {len(df_result)} features via Earth Engine")
    #             return df_result
                
    #         except ee.EEException as e:
    #             backoff = min(2 ** attempt, 60)
    #             logger.warning(f"EE error in batch {batch_idx + 1} (attempt {attempt+1}), retrying in {backoff}s: {e}")
    #             time.sleep(backoff)
    #         except Exception as e:
    #             logger.error(f"General error in batch {batch_idx + 1} (attempt {attempt+1}): {e}")
    #             if attempt < MAX_RETRIES - 1:
    #                 time.sleep(2 ** attempt)
        
    #     raise RuntimeError(f"Failed to process batch {batch_idx + 1} after {MAX_RETRIES} attempts")
    

    def _process_ee_feature_collection(self, feature_collection: ee.FeatureCollection, 
                                 national_codes: Optional[List[str]], batch_idx: int) -> pd.DataFrame:
        """Process FeatureCollection using Earth Engine with retry logic"""
        
        for attempt in range(MAX_RETRIES):
            try:
                # Use whisp's existing function that handles all conversions
                df_result = whisp.whisp_stats_ee_to_df(
                    feature_collection=feature_collection,
                    external_id_column=None,
                    remove_geom=False,
                    national_codes=national_codes,
                    unit_type="ha"
                )

                # Use whisp's existing function that handles all conversions
                # df_result = whisp.whisp_formatted_stats_ee_to_df(
                #     feature_collection=feature_collection,
                #     external_id_column=None,
                #     remove_geom=False,
                #     national_codes=national_codes,
                #     unit_type="ha"
                # )
                logger.info(f"Batch {batch_idx + 1}: Processed {len(df_result)} features via Earth Engine")
                return df_result
                
            except ee.EEException as e:
                backoff = min(2 ** attempt, 60)
                logger.warning(f"EE error in batch {batch_idx + 1} (attempt {attempt+1}), retrying in {backoff}s: {e}")
                time.sleep(backoff)
            except Exception as e:
                logger.error(f"General error in batch {batch_idx + 1} (attempt {attempt+1}): {e}")
                if attempt < MAX_RETRIES - 1:
                    time.sleep(2 ** attempt)
        
        raise RuntimeError(f"Failed to process batch {batch_idx + 1} after {MAX_RETRIES} attempts")

    
# High-volume batch processor with queue management
class WhispBatchQueue:
    """Queue-based processor for very high volume scenarios"""
    
    def __init__(self, features_per_batch=50, max_concurrent_batches=10):
        self.features_per_batch = features_per_batch
        self.max_concurrent_batches = max_concurrent_batches
        self.processor = OptimizedWhispProcessor(max_concurrent_batches, features_per_batch)
        
    def process_multiple_files(self, file_paths: List[str], national_codes: Optional[List[str]] = None) -> pd.DataFrame:
        """Process multiple files with intelligent batching"""
        
        all_results = []
        total_files = len(file_paths)
        
        logger.info(f"Processing {total_files} files with {self.features_per_batch} features per EE batch")
        
        for i, file_path in enumerate(file_paths):
            logger.info(f"Processing file {i+1}/{total_files}: {file_path}")
            
            try:
                result = self.processor.process_file_optimized(file_path, national_codes)
                if not result.empty:
                    result['source_file'] = file_path
                    all_results.append(result)
                    
            except Exception as e:
                logger.error(f"Failed to process {file_path}: {e}")
        
        # Combine all results
        if all_results:
            combined = pd.concat(all_results, ignore_index=True)
            logger.info(f"Total processed: {len(combined)} features from {len(all_results)} files")
            return combined
        else:
            return pd.DataFrame()


In [7]:
!pip show openforis-whisp

Name: openforis-whisp
Version: 2.0.0a6
Summary: Whisp (What is in that plot) is an open-source solution which helps to produce relevant forest monitoring information and support compliance with deforestation-related regulations.
Home-page: 
Author: Andy Arnell
Author-email: and.arnell@fao.org
License: MIT
Location: c:\Users\Arnell\Documents\GitHub\whisp\.venv\Lib\site-packages
Requires: country_converter, earthengine-api, geojson, geopandas, ipykernel, numpy, pandas, pandera, pydantic-core, python-dotenv, rsa, shapely
Required-by: 


In [8]:
folder_path = (r"C:\Users\Arnell\Downloads\processing_tests")  # Replace with your folder path

In [9]:
GEOJSON_EXAMPLE_FILEPATH = folder_path+"/random_polygons.geojson"

# Define bounds from the provided Earth Engine geometry
# area in Ghana 
bounds = [ 
    -3.04548260909834,  # min_lon
    5.253961384163733,  # min_lat
    -1.0179939534016594,  # max_lon
    7.48307210714245    # max_lat
]

# area in China
# bounds = [
#     103.44831497309737,  # min_lon
#     25.686366665187148,  # min_lat
#     109.57868606684737,  # max_lon
#     28.79200348254393    # max_lat
# ]

In [12]:
# random_geojson = whisp.create_geojson(
random_geojson = create_geojson(
    bounds, 
    num_polygons=5000, 
    min_area_ha=5, 
    max_area_ha=10, 
    min_number_vert=90, 
    max_number_vert=100)

GEOJSON_EXAMPLE_FILEPATH = folder_path + "/random_polygons.geojson"

import json
# Save the GeoJSON to a file
with open(GEOJSON_EXAMPLE_FILEPATH, 'w') as f:
    json.dump(random_geojson, f)

# Use example Whisp inputs (optional)
# GEOJSON_EXAMPLE_FILEPATH = whisp.get_example_data_path("geojson_example.geojson")


# Add IDs to your existing GeoJSON file

# #Save to a new file (instead of overwriting)
# # whisp.reformat_geojson_properties(
# whisp.reformat_geojson_properties(
    
#     geojson_path=GEOJSON_EXAMPLE_FILEPATH, 
#     id_field="internal_id",
#     output_path=folder_path + "/random_polygons_with_ids.geojson",
#     remove_properties=True
# )

In [13]:

# Example usage with controlled batch sizes
if __name__ == "__main__":
    
    # Configure batch size based on your data characteristics
    FEATURES_PER_EE_REQUEST = 25  # Small batches for complex geometries
    MAX_CONCURRENT_EE_REQUESTS = 20  # Conservative for quota management
    
    # Initialize processor
    processor = OptimizedWhispProcessor(
        max_concurrent=MAX_CONCURRENT_EE_REQUESTS,
        features_per_batch=FEATURES_PER_EE_REQUEST
    )
    
    # Process file with controlled batching
    try:
        # GEOJSON_EXAMPLE_FILEPATH = whisp.get_example_data_path("geojson_example.geojson")
        
        logger.info(f"Processing with {FEATURES_PER_EE_REQUEST} features per Earth Engine request")
        logger.info(f"Maximum {MAX_CONCURRENT_EE_REQUESTS} concurrent requests")
        
        result_df = processor.process_file_optimized(
            GEOJSON_EXAMPLE_FILEPATH, 
            national_codes=["br", "co"]
        )
        
        if not result_df.empty:
            print(f"Success! Processed {len(result_df)} features")
            print("\nFirst 5 rows:")
            print(result_df.head())
            
            # Save results
            result_df.to_csv("optimized_whisp_results.csv", index=False)
            logger.info("Results saved to optimized_whisp_results.csv")
        else:
            print("No results produced")
            
        print(f"Processing stats: {processor.processing_stats}")
        
    except Exception as e:
        logger.error(f"Processing failed: {e}")

2025-09-17 23:33:58,985 - INFO - Processing with 25 features per Earth Engine request
2025-09-17 23:33:58,985 - INFO - Maximum 20 concurrent requests
2025-09-17 23:34:03,241 - INFO - Processing 5000 features in batches of 25
2025-09-17 23:34:03,267 - INFO - Created 200 batches for Earth Engine processing
2025-09-17 23:34:07,941 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10
2025-09-17 23:34:08,175 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10


Whisp multiband image compiled
Whisp multiband image compiled
Whisp multiband image compiled
Whisp multiband image compiled
Whisp multiband image compiled
Whisp multiband image compiled
Whisp multiband image compiled
Whisp multiband image compiled


2025-09-17 23:34:10,971 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10


Whisp multiband image compiled
Whisp multiband image compiled


2025-09-17 23:34:11,097 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10


...

An error occurred during the conversion from EE to DataFrame: HTTPSConnectionPool(host='earthengine-highvolume.googleapis.com', port=443): Max retries exceeded with url: /v1/projects/234375012572/value:compute?prettyPrint=false&alt=json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001B40E754C80>: Failed to resolve 'earthengine-highvolume.googleapis.com' ([Errno 11001] getaddrinfo failed)"))


2025-09-18 08:40:59,269 - INFO - Batch 200: Processed 0 features via Earth Engine
2025-09-18 08:40:59,269 - INFO - Completed batch 200/200
2025-09-18 08:40:59,269 - INFO - Batch 197: Processed 0 features via Earth Engine
2025-09-18 08:40:59,284 - INFO - Completed batch 197/200
2025-09-18 08:40:59,378 - INFO - Successfully processed 1500 total features


An error occurred during the conversion from EE to DataFrame: HTTPSConnectionPool(host='earthengine-highvolume.googleapis.com', port=443): Max retries exceeded with url: /v1/projects/234375012572/value:compute?prettyPrint=false&alt=json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001B40E4A9910>: Failed to resolve 'earthengine-highvolume.googleapis.com' ([Errno 11001] getaddrinfo failed)"))
An error occurred during the conversion from EE to DataFrame: HTTPSConnectionPool(host='earthengine-highvolume.googleapis.com', port=443): Max retries exceeded with url: /v1/projects/234375012572/value:compute?prettyPrint=false&alt=json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001B40E4AB7A0>: Failed to resolve 'earthengine-highvolume.googleapis.com' ([Errno 11001] getaddrinfo failed)"))
Success! Processed 1500 features

First 5 rows:
                                                 geo         Admin_Level_1  \
0  {'type'

2025-09-18 08:41:00,165 - INFO - Results saved to optimized_whisp_results.csv


Processing stats: {'completed': 200, 'failed': 0, 'total': 0}


In [14]:
result_df  # Display first few rows of combined results

,geo,Admin_Level_1,Area,Centroid_lat,Centroid_lon,Cocoa_2023_FDaP,Cocoa_ETH,Cocoa_FDaP,Coffee_FDaP,Coffee_FDaP_2023,...,nBR_MapBiomas_col9_pasture_2020,nBR_MapBiomas_col9_pc_2020,nBR_MapBiomas_col9_silviculture_Brazil_2020,nBR_MapBiomas_col9_soy_2020,nBR_PRODES_deforestation_Brazil_after_2020,nBR_PRODES_deforestation_Brazil_before_2020,nCO_ideam_eufo_commission_2020,nCO_ideam_forest_2020,plotId,ProducerCountry
0,"{'type': 'Polygon', 'coordinates': [[[-1.64321...",Central Region,6.191,5.749666,-1.641660,4.389,4.601,5.260,0,0,...,0,0,0,0,0,0,0,0,1,GH
1,"{'type': 'Polygon', 'coordinates': [[[-1.97720...",Ashanti Region,8.624,6.717550,-1.975223,5.102,8.200,7.940,0,0,...,0,0,0,0,0,0,0,0,2,GH
2,"{'type': 'Polygon', 'coordinates': [[[-2.76007...",Western North Region,7.008,5.856680,-2.758233,5.751,6.741,6.738,0,0,...,0,0,0,0,0,0,0,0,3,GH
3,"{'type': 'Polygon', 'coordinates': [[[-2.97437...",Zanzan,9.092,7.218503,-2.972276,3.186,0.083,8.370,0,0,...,0,0,0,0,0,0,0,0,4,CI
4,"{'type': 'Polygon', 'coordinates': [[[-1.09460...",Ashanti Region,7.511,7.298841,-1.092791,0.213,0.000,0.592,0,0,...,0,0,0,0,0,0,0,0,5,GH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,"{'type': 'Polygon', 'coordinates': [[[-1.23163...",Central Region,10.746,5.653117,-1.229738,1.186,0.010,3.567,0,0,...,0,0,0,0,0,0,0,0,21,GH
1496,"{'type': 'Polygon', 'coordinates': [[[-1.29606...",Ashanti Region,7.113,7.144736,-1.294388,1.548,0.000,0.389,0,0,...,0,0,0,0,0,0,0,0,22,GH
1497,"{'type': 'Polygon', 'coordinates': [[[-1.19721...",Ashanti Region,9.236,6.908888,-1.195348,0.001,0.000,0.000,0,0,...,0,0,0,0,0,0,0,0,23,GH
1498,"{'type': 'Polygon', 'coordinates': [[[-2.35498...",Western Region,6.657,5.661819,-2.353256,5.032,4.833,5.850,0,0,...,0,0,0,0,0,0,0,0,24,GH


Classic Whisp

In [13]:
# Earth Engine and Common Libraries
import ee
from pathlib import Path

# Authenticate and initialize Earth Engine
try:
    ee.Initialize()  # Try to use existing credentials first
except Exception:
    ee.Authenticate()
    ee.Initialize()

In [14]:
import ee

# Reset Earth Engine completely
ee.Reset()

# Initialize with standard (normal) endpoint
ee.Initialize()

print("✅ Switched to STANDARD Earth Engine endpoint")

# Check the data module's base URL
print("EE Data Base URL:", ee.data._cloud_api_base_url)
print("EE API Base URL:", ee.data._api_base_url)

# Check if high-volume endpoint is configured
if 'highvolume' in str(ee.data._cloud_api_base_url):
    print("Using HIGH-VOLUME endpoint")
else:
    print("Using STANDARD endpoint")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


✅ Switched to STANDARD Earth Engine endpoint
EE Data Base URL: https://earthengine.googleapis.com
EE API Base URL: https://earthengine.googleapis.com/api
Using STANDARD endpoint


In [15]:
import openforis_whisp as whisp


In [16]:
!pip show openforis-whisp

Name: openforis-whisp
Version: 2.0.0a6
Summary: Whisp (What is in that plot) is an open-source solution which helps to produce relevant forest monitoring information and support compliance with deforestation-related regulations.
Home-page: 
Author: Andy Arnell
Author-email: and.arnell@fao.org
License: MIT
Location: c:\Users\Arnell\Documents\GitHub\whisp\.venv\Lib\site-packages
Requires: country_converter, earthengine-api, geojson, geopandas, ipykernel, numpy, pandas, pandera, pydantic-core, python-dotenv, rsa, shapely
Required-by: 


In [17]:
whisp = whisp.whisp_formatted_stats_geojson_to_df(GEOJSON_EXAMPLE_FILEPATH)

Reading GeoJSON file from: C:\Users\Arnell\Downloads\processing_tests\random_polygons.geojson
Whisp multiband image compiled
Creating schema for national_codes: None
[reformat.py | log_missing_columns() | l.267] INFO: All columns from dataframe found in the schema.
[reformat.py | log_missing_columns() | l.271] WARNING: The following columns in the schema did not match any columns from the results dataframe: 
external_id
